In [1]:
import pandas as pd
pd.set_option('display.max_rows', 500)
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import matplotlib
from matplotlib.pyplot import figure
import seaborn as sns

%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (12,8)
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, confusion_matrix, roc_curve, accuracy_score, f1_score, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.preprocessing import RobustScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.linear_model import LinearRegression

from catboost import CatBoostClassifier
from category_encoders import TargetEncoder

import lightgbm as lgb
from sklearn.cluster import KMeans
from sklearn.ensemble import IsolationForest

from tqdm import tqdm

from category_encoders import TargetEncoder

from pytorch_tabnet.tab_model import TabNetClassifier
import torch

In [59]:
data = pd.read_csv('data_prepared_to_AI_11')
test = pd.read_csv('test_dataset_hackathon_mkb.csv', sep=';', encoding='cp1251')

In [78]:
train_data = data.query('sample == 1').drop(['sample'], axis=1)
test_data = data.query('sample == 0').drop(['sample'], axis=1)
test_data = test_data.drop(['TARGET'], axis=1)
#train_data = train_data.drop_duplicates(subset = [i for i in train_data.columns if i != 'id_contract']).reset_index(drop=True)

y = train_data['TARGET'].values            
X = train_data.drop(['TARGET'], axis=1)

In [79]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(X)
data_train = scaler.transform(X)
data_test = scaler.transform(test_data)
#X_train, (X_test), y_train, y_test = train_test_split(data_train, train_data_y, test_size=0.2, random_state=13)
X_train, X_test, y_train, y_test = data_train[:14000,], data_train[14000:,], y[:14000,], y[14000:,]

In [80]:
X_train = X_train
y_train = y_train.squeeze()
X_test  = X_test

In [84]:
classifier = TabNetClassifier(verbose=1,seed=42)
classifier.fit(X_train=X_train, y_train=y_train, eval_set=[(X_test, y_test)], patience=5)

predictions = classifier.predict_proba(X_test)[:,1]

Device used : cuda
epoch 0  | loss: 0.86959 | val_0_auc: 0.6879  |  0:00:00s
epoch 1  | loss: 0.67186 | val_0_auc: 0.58743 |  0:00:01s
epoch 2  | loss: 0.60403 | val_0_auc: 0.73742 |  0:00:02s
epoch 3  | loss: 0.5273  | val_0_auc: 0.85056 |  0:00:03s
epoch 4  | loss: 0.47781 | val_0_auc: 0.88166 |  0:00:04s
epoch 5  | loss: 0.44434 | val_0_auc: 0.87845 |  0:00:05s
epoch 6  | loss: 0.41523 | val_0_auc: 0.87387 |  0:00:06s
epoch 7  | loss: 0.38727 | val_0_auc: 0.84222 |  0:00:07s
epoch 8  | loss: 0.36425 | val_0_auc: 0.89117 |  0:00:07s
epoch 9  | loss: 0.33923 | val_0_auc: 0.8466  |  0:00:08s
epoch 10 | loss: 0.31304 | val_0_auc: 0.88142 |  0:00:09s
epoch 11 | loss: 0.2871  | val_0_auc: 0.91012 |  0:00:10s
epoch 12 | loss: 0.26024 | val_0_auc: 0.92513 |  0:00:11s
epoch 13 | loss: 0.24706 | val_0_auc: 0.92311 |  0:00:11s
epoch 14 | loss: 0.23463 | val_0_auc: 0.91959 |  0:00:12s
epoch 15 | loss: 0.22386 | val_0_auc: 0.9152  |  0:00:13s
epoch 16 | loss: 0.21132 | val_0_auc: 0.9249  |  0:00

In [ ]:
fpr, tpr, _ = roc_curve(y_test, test_pred)
auc = roc_auc_score(y_test, test_pred)
accuracy = accuracy_score(y_test, test_pred_bin)
f1 = f1_score(y_test, test_pred_bin)
precision =  precision_score(y_test, test_pred_bin)
recall = recall_score(y_test, test_pred_bin)

print('FP, TP              :', "%0.2f" % sum(fpr), "%0.2f" % sum(tpr))
print('ROC_AUC_SCORE       :', "%0.2f" % auc)
print('accuracy            :', "%0.2f" % accuracy)
print('precision           :', "%0.2f" % precision)
print('recall              :', "%0.2f" % recall)
print('f1                  :', "%0.2f" % f1)

print('val, train  AUC     :', "%0.2f" % roc_auc_score(y_test, test_pred), "%0.2f" % roc_auc_score(y_train, RF.predict_proba(X_train)[:,1]))
print('val, train  AUC_2   :', "%0.2f" % roc_auc_score(y_test, test_pred_bin), "%0.2f" % roc_auc_score(y_train, RF.predict(X_train)))
print(confusion_matrix(y_test, test_pred_bin))
print(classification_report(y_test, RF.predict(X_test)))

In [85]:
predictions_bin = classifier.predict(X_test)
print(classification_report(y_test, predictions_bin))
print(confusion_matrix(y_test, predictions_bin))

              precision    recall  f1-score   support

           0       0.82      0.95      0.88      2497
           1       0.88      0.63      0.74      1394

    accuracy                           0.84      3891
   macro avg       0.85      0.79      0.81      3891
weighted avg       0.84      0.84      0.83      3891

[[2374  123]
 [ 512  882]]


In [77]:
roc_auc_score(y_test, predictions)

0.9278038955211102

In [56]:
y_test.shape

(3891,)

In [57]:
print(confusion_matrix(y_test, np.argmax(predictions_bin, axis=-1)))

[[1363 1134]
 [ 331 1063]]


In [67]:
y_train

array([0, 0, 0, ..., 0, 0, 1])

In [71]:
predictions_bin

array([1, 1, 0, ..., 0, 0, 0])